In [11]:
print("hello world")

hello world


In [2]:
pip install requests

  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
  Using cached urllib3-2.3.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached certifi-2024.12.14-py3-none-any.whl.metadata (2.3 kB)
Using cached requests-2.32.3-py3-none-any.whl (64 kB)
Using cached certifi-2024.12.14-py3-none-any.whl (164 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.1/196.1 kB 4.2 MB/s eta 0:00:0000:01
Using cached idna-3.10-py3-none-any.whl (70 kB)
Using cached urllib3-2.3.0-py3-none-any.whl (128 kB)

[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
import requests

url = "https://storage.googleapis.com/mediapipe-models/face_detector/blaze_face_short_range/float16/1/blaze_face_short_range.tflite"
output_path = "detector.tflite"

response = requests.get(url)
if response.status_code == 200:
    with open(output_path, "wb") as f:
        f.write(response.content)
    print(f"Downloaded to {output_path}")
else:
    print(f"Failed to download. HTTP Status Code: {response.status_code}")



Downloaded to detector.tflite


In [4]:
pip install mediapipe

  Using cached absl_py-2.1.0-py3-none-any.whl.metadata (2.3 kB)
  Using cached attrs-24.3.0-py3-none-any.whl.metadata (11 kB)
  Using cached flatbuffers-24.12.23-py2.py3-none-any.whl.metadata (876 bytes)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 5.3 MB/s eta 0:00:00
  Using cached opencv_contrib_python-4.10.0.84-cp37-abi3-macosx_11_0_arm64.whl.metadata (20 kB)
  Using cached protobuf-4.25.5-cp37-abi3-macosx_10_9_universal2.whl.metadata (541 bytes)
  Using cached sounddevice-0.5.1-py3-none-macosx_10_6_x86_64.macosx_10_6_universal2.whl.metadata (1.4 kB)
  Using cached opt_einsum-3.4.0-py3-none-any.whl.metadata (6.3 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 3.3 MB/s eta 0:00:00
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.1/165.1 kB 5.1 MB/s eta 0:00:00
  Using cached pyparsing-3.2.1-py3-none-any.whl.metadata (5.0 kB)
  Using cached pycparser-2.22-py3-none-any.whl.metadata (943 bytes)

In [6]:
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
import cv2

detection_results = []  # Global variable to store results for annotation
def print_result(result, output_image, timestamp_ms):
    global detection_results
    detection_results = result.detections  # Store detections for annotation

# Visualization function
def visualize(image, detections):
    """Draw bounding boxes on the image."""
    annotated_image = image.copy()
    height, width, _ = image.shape
    for detection in detections:
        bbox = detection.bounding_box
        start_point = (bbox.origin_x, bbox.origin_y)
        end_point = (bbox.origin_x + bbox.width, bbox.origin_y + bbox.height)
        # Draw the bounding box
        cv2.rectangle(annotated_image, start_point, end_point, (0, 255, 0), 2)
    return annotated_image



# Load the MediaPipe model
BaseOptions = mp.tasks.BaseOptions
FaceDetector = mp.tasks.vision.FaceDetector
FaceDetectorOptions = mp.tasks.vision.FaceDetectorOptions
VisionRunningMode = mp.tasks.vision.RunningMode

# Path to the downloaded model
model_path = "detector.tflite"

# Set options for live stream
options = FaceDetectorOptions(
    base_options=BaseOptions(model_asset_path=model_path),
    running_mode=VisionRunningMode.LIVE_STREAM,
    result_callback=print_result
)



In [5]:
#from fer import FER

In [2]:
import cv2
import mediapipe as mp
# from deepface import DeepFace

# Initialize MediaPipe Face Mesh
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=False, max_num_faces=1, min_detection_confidence=0.5)


# Initialize MediaPipe Pose
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()
mp_drawing = mp.solutions.drawing_utils



# Initialize FER
#emotion_detector = FER(mtcnn=True)

# Access the webcam
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("Error: Unable to access the camera.")
else:
    print("Camera is on. Press 'q' to quit.")

    while True:
        ret, frame = cap.read()
        if not ret:
            print("Error: Unable to read frame.")
            break

        # Convert the frame to RGB for MediaPipe and FER
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Detect face mesh
        results_face = face_mesh.process(rgb_frame)
        if results_face.multi_face_landmarks:
            for face_landmarks in results_face.multi_face_landmarks:
                # Draw the facial landmarks on the frame
                mp.solutions.drawing_utils.draw_landmarks(
                    frame, face_landmarks, mp_face_mesh.FACEMESH_TESSELATION,
                    mp.solutions.drawing_styles.get_default_face_mesh_tesselation_style())


        # try:
        #     result = DeepFace.analyze(frame, actions=['emotion'], enforce_detection=False)
        #     # Display dominant emotion on the frame
        #     dominant_emotion = max(result['emotion'], key=result['emotion'].get)
        #     cv2.putText(frame, dominant_emotion, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        # except Exception as e:
        #     print("No face detected:", str(e))
    

        # if results_face.multi_face_landmarks:
        #     for face_landmarks in results_face.multi_face_landmarks:
        #         # Example: Detect smile
        #         landmarks = face_landmarks.landmark
        #         # Calculate distances
        #         left_corner = landmarks[61]  # Left corner of mouth
        #         right_corner = landmarks[291]  # Right corner of mouth
        #         upper_lip = landmarks[13]  # Upper lip
        #         lower_lip = landmarks[14]  # Lower lip

        #         # Horizontal distance (mouth width)
        #         mouth_width = ((right_corner.x - left_corner.x) ** 2 +
        #                     (right_corner.y - left_corner.y) ** 2) ** 0.5
        #         # Vertical distance (mouth openness)
        #         mouth_height = ((upper_lip.x - lower_lip.x) ** 2 +
        #                         (upper_lip.y - lower_lip.y) ** 2) ** 0.5

        #         # Define a smile condition
        #         if mouth_width / mouth_height > 3:  # Adjust the threshold as needed
        #             cv2.putText(frame, "Smiling", (50, 50),
        #                         cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

        # Process the frame to detect pose
        results_pose = pose.process(rgb_frame)

        # Draw landmarks on the frame
        if results_pose.pose_landmarks:
            mp_drawing.draw_landmarks(
                frame, results_pose.pose_landmarks, mp_pose.POSE_CONNECTIONS
            )

        # Display the frame
        cv2.imshow("MediaPipe Pose", frame)



        # Detect emotions
        # emotions = emotion_detector.detect_emotions(frame)
        # if emotions:
        #     dominant_emotion, emotion_probabilities = emotions[0]['emotions'].items(), emotions[0]['emotions']
        #     emotion_text = f"Emotion: {max(emotion_probabilities, key=emotion_probabilities.get)}"
        #     cv2.putText(frame, emotion_text, (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

        # Display the frame
        cv2.imshow('Face Mesh & Emotion Detection', frame)

        # Exit when 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            print("Turning off the camera.")
            break

# Release resources
cap.release()
cv2.destroyAllWindows()


I0000 00:00:1736291397.193483  228414 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.3), renderer: Apple M4
I0000 00:00:1736291397.199847  228414 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.3), renderer: Apple M4
W0000 00:00:1736291397.205173  229901 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1736291397.212812  229901 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1736291397.265896  229910 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1736291397.273748  229914 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


Camera is on. Press 'q' to quit.
Turning off the camera.


In [ ]:
#old code just keeping to reference

import cv2



# Access the webcam
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("Error: Unable to access the camera.")
    exit()

# Initialize a manual timestamp counter
current_timestamp_ms = 0
timestamp_increment = 33  # Adjust for your camera's frame rate

# Create the face detector instance
with FaceDetector.create_from_options(options) as detector:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            print("Failed to capture frame.")
            break

        # Resize frame to improve performance
        frame = cv2.resize(frame, (640, 480))

        # Convert frame to RGB
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Convert to MediaPipe Image
        mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=frame_rgb)

        try:
            # Increment and send manual timestamp
            current_timestamp_ms += timestamp_increment
            detector.detect_async(mp_image, current_timestamp_ms)
        except ValueError as e:
            print(f"Error in detection: {e}")
            break


        # Draw bounding boxes on the frame using the latest results
        annotated_frame = visualize(frame, detection_results)

        # Display the annotated frame
        cv2.imshow("Face Detection", annotated_frame)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):  # Exit on 'q' key press
            break

cap.release()
cv2.destroyAllWindows()

I0000 00:00:1733909744.847078 2698198 gl_context.cc:357] GL version: 2.1 (2.1 ATI-4.14.1), renderer: AMD Radeon Pro 555 OpenGL Engine
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1733909744.874495 2698901 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


: 